In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
%matplotlib inline
import scipy.stats as stats
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn import metrics
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices

C:\Users\hp\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data=pd.read_csv('C:\\Users\\hp\\Desktop\\analytixlabs\\Ml case study\\network intru\\NSL_Dataset\\Train.txt',header=None)

In [3]:
data.columns=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land", "wrong_fragment","urgent","hot","num_failed_logins","logged_in", "num_compromised","root_shell","su_attempted","num_root","num_file_creations", "num_shells","num_access_files","num_outbound_cmds","is_host_login", "is_guest_login","count","srv_count","serror_rate", "srv_serror_rate", "rerror_rate","srv_rerror_rate","same_srv_rate", "diff_srv_rate",
             "srv_diff_host_rate","dst_host_count","dst_host_srv_count","dst_host_same_srv_rate", "dst_host_diff_srv_rate","dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate", "dst_host_rerror_rate","dst_host_srv_rerror_rate","attack", "last_flag"]

In [4]:
data.loc[(data.attack =='normal'),'attack']= 0

data.loc[(data.attack=='neptune')|(data.attack=='back')|(data.attack=='land')|(data.attack=='pod')|
            (data.attack=='smurf')|(data.attack=='teardrop')|(data.attack=='apache2')|(data.attack=='udpstorm')|
            (data.attack=='processtable')|(data.attack=='worm')|(data.attack== 'mailbomb'),'attack'] = 1

data.loc[(data.attack=='satan')|(data.attack=='ipsweep')|(data.attack=='nmap')|(data.attack=='portsweep')|
         (data.attack=='mscan')|(data.attack=='saint'),'attack'] = 2

data.loc[(data.attack=='guess_passwd')|(data.attack=='ftp_write')|(data.attack=='imap')|(data.attack=='phf')|
         (data.attack=='multihop')|(data.attack=='warezmaster')|(data.attack=='warezclient')|(data.attack=='spy')|
         (data.attack=='xlock')|(data.attack=='xsnoop')|(data.attack=='snmpguess')|(data.attack=='snmpgetattack')|
         (data.attack=='httptunnel')|(data.attack=='sendmail')|(data.attack=='named'),'attack'] = 3

data.loc[(data.attack=='buffer_overflow')|(data.attack=='loadmodule')|(data.attack=='rootkit')|(data.attack=='perl')|
         (data.attack=='sqlattack')|(data.attack=='xterm')|(data.attack=='ps'),'attack'] = 4


In [5]:
data.attack=data.attack.astype('int')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 43 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     125973 non-null  int64  
 1   protocol_type                125973 non-null  object 
 2   service                      125973 non-null  object 
 3   flag                         125973 non-null  object 
 4   src_bytes                    125973 non-null  int64  
 5   dst_bytes                    125973 non-null  int64  
 6   land                         125973 non-null  int64  
 7   wrong_fragment               125973 non-null  int64  
 8   urgent                       125973 non-null  int64  
 9   hot                          125973 non-null  int64  
 10  num_failed_logins            125973 non-null  int64  
 11  logged_in                    125973 non-null  int64  
 12  num_compromised              125973 non-null  int64  
 13 

In [7]:
def continuous_var_summary(x):
    return pd.Series([x.count(), x.isnull().sum(), x.sum(), x.mean(), x.median(),  
                      x.std(), x.var(), x.min(), x.quantile(0.01), x.quantile(0.05),
                          x.quantile(0.10),x.quantile(0.25),x.quantile(0.50),x.quantile(0.75), 
                              x.quantile(0.90),x.quantile(0.95), x.quantile(0.99),x.max()], 
                  index = ['N', 'NMISS', 'SUM', 'MEAN','MEDIAN', 'STD', 'VAR', 'MIN', 'P1', 
                               'P5' ,'P10' ,'P25' ,'P50' ,'P75' ,'P90' ,'P95' ,'P99' ,'MAX'])

In [8]:
conti_var=data.loc[:,(data.dtypes=='int64')|(data.dtypes=='float64')|(data.dtypes=='int32')]

In [9]:
categ_var=data.loc[:,data.dtypes=='object']

In [10]:
conti_var.apply(lambda x:continuous_var_summary(x)).T

,N,NMISS,SUM,MEAN,MEDIAN,STD,VAR,MIN,P1,P5,P10,P25,P50,P75,P90,P95,P99,MAX
duration,125973.0,0.0,3.617247e+07,287.144650,0.00,2.604515e+03,6.783500e+06,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,4.00,9590.56,4.290800e+04
src_bytes,125973.0,0.0,5.740179e+09,45566.743000,44.00,5.870331e+06,3.446079e+13,0.0,0.0,0.00,0.00,0.00,44.00,276.00,848.00,1480.00,54540.00,1.379964e+09
dst_bytes,125973.0,0.0,2.491634e+09,19779.114421,0.00,4.021269e+06,1.617061e+13,0.0,0.0,0.00,0.00,0.00,0.00,516.00,3375.80,8314.00,25519.00,1.309937e+09
land,125973.0,0.0,2.500000e+01,0.000198,0.00,1.408607e-02,1.984174e-04,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.000000e+00
wrong_fragment,125973.0,0.0,2.858000e+03,0.022687,0.00,2.535300e-01,6.427745e-02,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.000000e+00
urgent,125973.0,0.0,1.400000e+01,0.000111,0.00,1.436603e-02,2.063827e-04,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.000000e+00
hot,125973.0,0.0,2.575000e+04,0.204409,0.00,2.149968e+00,4.622364e+00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.00,7.700000e+01
num_failed_logins,125973.0,0.0,1.540000e+02,0.001222,0.00,4.523914e-02,2.046580e-03,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5.000000e+00
logged_in,125973.0,0.0,4.985200e+04,0.395736,0.00,4.890101e-01,2.391308e-01,0.0,0.0,0.00,0.00,0.00,0.00,1.00,1.00,1.00,1.00,1.000000e+00
num_compromised,125973.0,0.0,3.517800e+04,0.279250,0.00,2.394204e+01,5.732214e+02,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,7.479000e+03


In [11]:
conti_var=conti_var.apply(lambda x:x.clip(lower=x.quantile(0.01),upper=x.quantile(0.99)))

In [12]:
def create_dummies(df, colname):
    col_dummies = pd.get_dummies(df[colname], prefix = colname, drop_first = True)
    df = pd.concat([df, col_dummies], axis = 1)
    df.drop(colname, axis = 1, inplace = True )
    return df

In [13]:
for c_feature in ['protocol_type','service','flag']:
    categ_var[c_feature] = categ_var[c_feature].astype('category')
    categ_var = create_dummies(categ_var, c_feature)

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
data2=pd.concat([conti_var,categ_var],axis=1)


In [15]:
data2.corr().to_excel('netcor.xlsx')

In [16]:
data2.drop(columns=['land',
'wrong_fragment',
'urgent',
'num_failed_logins',
'root_shell',
'su_attempted',
'num_root',
'num_file_creations',
'num_shells',
'num_access_files',
'num_outbound_cmds',
'is_host_login',
'is_guest_login',
'service_aol',
'service_harvest',
'service_http_2784',
'service_http_8001',
'service_red_i',
'service_tftp_u',
'service_tim_i',
'service_urh_i'],
inplace=True)

In [17]:
data2.shape

(125973, 100)

In [18]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [19]:
feature_col=data2.columns.difference(['attack'])

In [21]:
logreg=LogisticRegression(n_jobs=-1,solver='lbfgs')
rfe=RFE(logreg,n_features_to_select=25,verbose=10)

In [22]:
rfe.fit(data2[feature_col],data2.attack)
print(rfe.support_)

Fitting estimator with 99 features.


C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Fitting estimator with 98 features.


C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Fitting estimator with 97 features.


...

Fitting estimator with 27 features.


C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Fitting estimator with 26 features.


C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[False  True False False  True  True  True False False False  True False
  True False  True  True  True  True  True  True False False False False
 False False False  True  True False False  True  True False False False
 False False False False False False False  True False  True  True False
 False False  True False False False  True False False False False False
 False False False False False False False False False False False False
  True False False False False  True False False False False False False
 False False False  True False  True False False False False False False
 False False False]


In [20]:
x_var=data2[feature_col[rfe.support_]]

NameError: name 'rfe' is not defined

In [24]:
fdata=pd.concat([x_var,data2.attack],axis=1)

In [25]:
feature='+'.join(x_var.columns)

In [26]:
a,b=dmatrices( 'attack ~'+ feature,data=fdata,return_type='dataframe')

In [27]:
vif=pd.DataFrame()
vif['vif']= [variance_inflation_factor(b.values, i) for i in range(b.shape[1])]
vif["features"] = b.columns
vif

,vif,features
0,68.022888,Intercept
1,2.134266,diff_srv_rate
2,2.629832,dst_host_diff_srv_rate
3,8.201229,dst_host_rerror_rate
4,2.630486,dst_host_same_src_port_rate
5,2.819689,dst_host_srv_diff_host_rate
6,40.375379,dst_host_srv_serror_rate
7,6.797786,flag_REJ
8,1.816112,flag_RSTO
9,1.063032,flag_RSTOS0


In [28]:
vif[vif['vif']>5].features.values

array(['Intercept', 'dst_host_rerror_rate', 'dst_host_srv_serror_rate',
       'flag_REJ', 'flag_S0', 'protocol_type_tcp', 'same_srv_rate',
       'serror_rate'], dtype=object)

In [29]:
fdata.drop(columns=['dst_host_rerror_rate', 'dst_host_srv_serror_rate',
       'flag_REJ', 'flag_S0', 'protocol_type_tcp', 'same_srv_rate',
       'serror_rate'],inplace=True)

In [32]:
feature2='+'.join(fdata.columns.difference(['attack']))

In [33]:
feature2

'diff_srv_rate+dst_host_diff_srv_rate+dst_host_same_src_port_rate+dst_host_srv_diff_host_rate+flag_RSTO+flag_RSTOS0+flag_RSTR+flag_S1+num_compromised+service_domain_u+service_eco_i+service_ecr_i+service_ftp+service_http+service_other+service_private+service_telnet+service_urp_i'

In [34]:
a,b=dmatrices( 'attack ~'+ feature2,data=fdata,return_type='dataframe')

In [35]:
vif=pd.DataFrame()
vif['vif']= [variance_inflation_factor(b.values, i) for i in range(b.shape[1])]
vif["features"] = b.columns
vif

,vif,features
0,3.872193,Intercept
1,1.545757,diff_srv_rate
2,2.142262,dst_host_diff_srv_rate
3,2.180058,dst_host_same_src_port_rate
4,2.580094,dst_host_srv_diff_host_rate
5,1.025898,flag_RSTO
6,1.007827,flag_RSTOS0
7,1.261300,flag_RSTR
8,1.006550,flag_S1
9,1.049859,num_compromised


In [36]:
vif[vif['vif']>5].features.values

array([], dtype=object)

In [37]:
faeture_col2=fdata.columns.difference(['attack'])

In [38]:
train_X, test_X, train_y, test_y = train_test_split(fdata[faeture_col2],
                    fdata['attack'], test_size = 0.3, random_state = 123)

In [39]:
log_model = LogisticRegression(random_state = 123).fit( train_X, train_y )

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [40]:
log_model.score(train_X, train_y )

0.8804957984146244

In [42]:
log_model.score(test_X, test_y )

0.8796835309060118

In [43]:
c=log_model.predict(train_X)
d=log_model.predict(test_X)

In [44]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score

In [45]:
confusion_matrix(train_y,c,[1,0])

array([[30447,  1379],
       [ 6823, 39456]], dtype=int64)

In [46]:
confusion_matrix(test_y,d,[1,0])

array([[13130,   614],
       [ 2910, 16854]], dtype=int64)

In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [53]:
param_grid = {'n_estimators':[120,125], 
                 'max_features':[3,4]}

In [54]:
tree = GridSearchCV(RandomForestClassifier(random_state=123), param_grid, 
                        cv =10,verbose=10).fit( train_X, train_y )

Fitting 10 folds for each of 4 candidates, totalling 40 fits
[CV] max_features=3, n_estimators=120 ................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .... max_features=3, n_estimators=120, score=0.987, total=   6.0s
[CV] max_features=3, n_estimators=120 ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.9s remaining:    0.0s


[CV] .... max_features=3, n_estimators=120, score=0.987, total=   6.6s
[CV] max_features=3, n_estimators=120 ................................


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   48.7s remaining:    0.0s


[CV] .... max_features=3, n_estimators=120, score=0.989, total=   6.0s
[CV] max_features=3, n_estimators=120 ................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   54.7s remaining:    0.0s


[CV] .... max_features=3, n_estimators=120, score=0.987, total=   5.9s
[CV] max_features=3, n_estimators=125 ................................
[CV] .... max_features=3, n_estimators=125, score=0.987, total=   6.5s
[CV] max_features=3, n_estimators=125 ................................
[CV] .... max_features=3, n_estimators=125, score=0.987, total=   6.4s
[CV] max_features=3, n_estimators=125 ................................
[CV] .... max_features=3, n_estimators=125, score=0.986, total=   6.4s
[CV] max_features=3, n_estimators=125 ................................
[CV] .... max_features=3, n_estimators=125, score=0.989, total=   6.3s
[CV] max_features=3, n_estimators=125 ................................
[CV] .... max_features=3, n_estimators=125, score=0.988, total=   6.3s
[CV] max_features=3, n_estimators=125 ................................
[CV] .... max_features=3, n_estimators=125, score=0.985, total=   6.3s
[CV] max_features=3, n_estimators=125 ................................
[CV] .

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  4.4min finished


In [55]:
tree.best_score_

0.9876163799457933

In [56]:
tree.best_params_

{'max_features': 3, 'n_estimators': 125}

In [57]:
tree1=tree.best_estimator_.fit(train_X,train_y)

In [58]:
tree1.score(test_X,test_y)

0.9868226079593565

# now importing and pridicting on actal testing dataset.


In [89]:
tdata=pd.read_csv('C:\\Users\\hp\\Desktop\\analytixlabs\\Ml case study\\network intru\\NSL_Dataset\\Test.txt',header=None)

In [90]:
tdata.columns=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land", "wrong_fragment","urgent","hot","num_failed_logins","logged_in", "num_compromised","root_shell","su_attempted","num_root","num_file_creations", "num_shells","num_access_files","num_outbound_cmds","is_host_login", "is_guest_login","count","srv_count","serror_rate", "srv_serror_rate", "rerror_rate","srv_rerror_rate","same_srv_rate", "diff_srv_rate",
             "srv_diff_host_rate","dst_host_count","dst_host_srv_count","dst_host_same_srv_rate", "dst_host_diff_srv_rate","dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate", "dst_host_rerror_rate","dst_host_srv_rerror_rate","attack", "last_flag"]

In [91]:
tdata.loc[(tdata.attack =='normal'),'attack']= 0

tdata.loc[(tdata.attack=='neptune')|(tdata.attack=='back')|(tdata.attack=='land')|(tdata.attack=='pod')|
            (tdata.attack=='smurf')|(tdata.attack=='teardrop')|(tdata.attack=='apache2')|(tdata.attack=='udpstorm')|
            (tdata.attack=='processtable')|(tdata.attack=='worm')|(tdata.attack== 'mailbomb'),'attack'] = 1

tdata.loc[(tdata.attack=='satan')|(tdata.attack=='ipsweep')|(tdata.attack=='nmap')|(tdata.attack=='portsweep')|
         (tdata.attack=='mscan')|(tdata.attack=='saint'),'attack'] = 2

tdata.loc[(tdata.attack=='guess_passwd')|(tdata.attack=='ftp_write')|(tdata.attack=='imap')|(tdata.attack=='phf')|
         (tdata.attack=='multihop')|(tdata.attack=='warezmaster')|(tdata.attack=='warezclient')|(tdata.attack=='spy')|
         (tdata.attack=='xlock')|(tdata.attack=='xsnoop')|(tdata.attack=='snmpguess')|(tdata.attack=='snmpgetattack')|
         (tdata.attack=='httptunnel')|(tdata.attack=='sendmail')|(tdata.attack=='named'),'attack'] = 3

tdata.loc[(tdata.attack=='buffer_overflow')|(tdata.attack=='loadmodule')|(tdata.attack=='rootkit')|(tdata.attack=='perl')|
         (tdata.attack=='sqlattack')|(tdata.attack=='xterm')|(tdata.attack=='ps'),'attack'] = 4

In [92]:
tdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22544 entries, 0 to 22543
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     22544 non-null  int64  
 1   protocol_type                22544 non-null  object 
 2   service                      22544 non-null  object 
 3   flag                         22544 non-null  object 
 4   src_bytes                    22544 non-null  int64  
 5   dst_bytes                    22544 non-null  int64  
 6   land                         22544 non-null  int64  
 7   wrong_fragment               22544 non-null  int64  
 8   urgent                       22544 non-null  int64  
 9   hot                          22544 non-null  int64  
 10  num_failed_logins            22544 non-null  int64  
 11  logged_in                    22544 non-null  int64  
 12  num_compromised              22544 non-null  int64  
 13  root_shell      

In [93]:
tdata.attack=tdata.attack.astype('int')

In [109]:
conti_var_test=tdata.loc[:,(tdata.dtypes=='int64')|(tdata.dtypes=='float64')|(tdata.dtypes=='int32')]

In [95]:
categ_var_test=tdata.loc[:,tdata.dtypes=='object']

In [96]:
categ_var_test

,protocol_type,service,flag
0,tcp,private,REJ
1,tcp,private,REJ
2,tcp,ftp_data,SF
3,icmp,eco_i,SF
4,tcp,telnet,RSTO
...,...,...,...
22539,tcp,smtp,SF
22540,tcp,http,SF
22541,tcp,http,SF
22542,udp,domain_u,SF


In [97]:
conti_var_test=conti_var_test.apply(lambda x:x.clip(lower=x.quantile(0.01),upper=x.quantile(0.99)))

In [98]:
for c_feature in ['protocol_type','service','flag']:
    categ_var_test[c_feature] = categ_var_test[c_feature].astype('category')
    categ_var_test = create_dummies(categ_var_test, c_feature)

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [110]:
datatst=pd.concat([conti_var_test,categ_var_test],axis=1)

In [111]:
train_X.columns

Index(['diff_srv_rate', 'dst_host_diff_srv_rate',
       'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
       'flag_RSTO', 'flag_RSTOS0', 'flag_RSTR', 'flag_S1', 'num_compromised',
       'service_domain_u', 'service_eco_i', 'service_ecr_i', 'service_ftp',
       'service_http', 'service_other', 'service_private', 'service_telnet',
       'service_urp_i'],
      dtype='object')

In [112]:
x_var=datatst[train_X.columns]

In [113]:
x_var.columns

Index(['diff_srv_rate', 'dst_host_diff_srv_rate',
       'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
       'flag_RSTO', 'flag_RSTOS0', 'flag_RSTR', 'flag_S1', 'num_compromised',
       'service_domain_u', 'service_eco_i', 'service_ecr_i', 'service_ftp',
       'service_http', 'service_other', 'service_private', 'service_telnet',
       'service_urp_i'],
      dtype='object')

In [114]:
y_var=datatst.attack

In [115]:
y_var

0        1
1        1
2        0
3        2
4        2
        ..
22539    0
22540    0
22541    1
22542    0
22543    2
Name: attack, Length: 22544, dtype: int32

In [116]:
pred_y=tree1.predict(x_var)

In [117]:
metrics.accuracy_score(pred_y,y_var)

0.7295067423704755

# after trying various techniques i am getting max accuracy with Random forest of 72% of testing data set